In [49]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


## Initialization

In [50]:
import numpy as np
import random
from scipy.spatial import distance as d
import os
import math
import pandas as pd
import time
import json
import PIL

TEST_DIR = '/content/gdrive/My Drive/test-folder/'

LEAF_FOLDER = '/content/gdrive/My Drive/[MIRCV]FoodWebSearch/antonio-tests'

TEST_FILE_1 = os.path.join(TEST_DIR, "test-tree-construction-data.npy")
TEST_FILE_2 = os.path.join(TEST_DIR, "test-tree-construction-data-2.npy")
TEST_FILE_3 = os.path.join(TEST_DIR, "test-tree-construction-data-3.npy")


TEST_PATH = "/content/gdrive/My Drive/[MIRCV]FoodWebSearch/antonio-tests"

FEATURES_PATH = "/content/gdrive/My Drive/[MIRCV]FoodWebSearch/features.csv"
FEATURES_PATH_TEST_1 = "/content/gdrive/My Drive/[MIRCV]FoodWebSearch/antonio-tests/features-test-1.npy"
FEATURES_NAMES_TEST_1 = "/content/gdrive/My Drive/[MIRCV]FoodWebSearch/antonio-tests/features-names-test-1.npy"

FEATURES_PATH_TEST_2 = "/content/gdrive/My Drive/[MIRCV]FoodWebSearch/antonio-tests/features-test-2.npy"
FEATURES_NAMES_TEST_2 = "/content/gdrive/My Drive/[MIRCV]FoodWebSearch/antonio-tests/features-names-test-2.npy"

FEATURES_PATH_TEST_3 = "/content/gdrive/My Drive/[MIRCV]FoodWebSearch/antonio-tests/features-test-3.npy"
FEATURES_NAMES_TEST_3 = "/content/gdrive/My Drive/[MIRCV]FoodWebSearch/antonio-tests/features-names-test-3.npy"


ID_DEPLOYMENT = "/content/gdrive/MyDrive/[MIRCV]FoodWebSearch/id.npy"
FEATURES_DEPLOYMENT = "/content/gdrive/MyDrive/[MIRCV]FoodWebSearch/features.npy"

FINE_TUNED_ID = '/content/gdrive/MyDrive/[MIRCV]FoodWebSearch/deployment/ft_id.npy'
FINE_TUNED_FEATURES = '/content/gdrive/MyDrive/[MIRCV]FoodWebSearch/deployment/ft_features.npy'

MN_ID = '/content/gdrive/MyDrive/[MIRCV]FoodWebSearch/deployment/mn_id.npy'
MN_FEATURES = '/content/gdrive/MyDrive/[MIRCV]FoodWebSearch/deployment/mn_features.npy'

INDEX_DIR = "/content/gdrive/MyDrive/[MIRCV]FoodWebSearch/deployment"

DATA_PATH = "/content/gdrive/MyDrive/food-distractor-dataset/food-101"

# Display results
def display_results(results):
  for filename, distance in results:
    print('File: {} - Distance: {:.6f}'.format(filename, distance))
    filepath = os.path.join(DATA_PATH,filename)
    image = PIL.Image.open(filepath)
    image.thumbnail((224,224))
    display(image)

# Create Tree Data Structure

##The Node
The class Node represents a node in the tree. 
We can have two types of nodes:
- internal node
- leaf node

All the nodes have an id, in order to store the tree on disk, preserving its structure.

####Internal Node
In the case of an internal node, the class has the following parameters:
- reference to the right child
- reference to the left child
- the value of the node, it's the pivot in this application: a numpy array that represent a point in the space
- the median: the value of the median is computed on the sorted distances between the pivot and the other values of the set

####Leaf Node

In the case of a leaf node, the class has the following fields:
- the pivot
- the median
- reference to the file in which are stored the objects on the left
- reference to the file in which are stored the objects on the right

### Distances
The choice of the distances in this VPTree is very important because in the case of a lot of dimensions (e.g. hundreds of dimensions like in this case) the euclidean distance is not the right choice:
- it's very influeced by the number of dimensions in the number of overlapping regions created
- it's not suitable in the case of input features not normalized

With Cosine similarity, the results they are slightly better: with the same height and the same number of data, the file accessed are less. Anyway, the overlapping regions remains still a lot.
The VP_Tree class provides the possibility to choose between this two kind of distances.

Trials has been done also with Hamming distance:
- the number of file visited is very small
- the speed is very high
- the results are not so accurate.


In [51]:
class NumpyEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, np.ndarray):
            return obj.tolist()
        if isinstance(obj, None):
            return ""
        return json.JSONEncoder.default(self, obj)

In [52]:
class Node:

  def __init__(self, id, is_leaf, **kwargs):
    self.parent = kwargs.get("parent", None)
    self.id = id
    self.is_leaf = is_leaf
    self.pivot = kwargs.get("pivot", None)
    self.median = kwargs.get("median", -1)
    if self.is_leaf:
      self.objects = kwargs.get("objects", [])
      self.file_path_s_1, self.file_path_s_2 = "", ""
    else:
      self.right = kwargs.get("right", None)
      self.left = kwargs.get("left", None)

  def set_parameters(self, pivot, median):
    self.pivot = pivot
    self.median = median

  def add_children(self, left, right):
    self.left = left
    self.right = right

  def add_objects(self, s_1, s_2):
    self.objects_left = s_1
    self.objects_right = s_2

  def save_leaf_objects_on_disk(self, file_path, s_1, s_2):
    self.file_path_s_1 = file_path + "_subset_1.npy"
    self.file_path_s_2 = file_path + "_subset_2.npy"
    np.save(self.file_path_s_1, np.array(s_1, dtype=object))
    np.save(self.file_path_s_2, np.array(s_2, dtype=object))

  def load_objects_from_disk(self, left=True, right=True):
    if left and not right:
      result = np.load(self.file_path_s_1, allow_pickle=True)
      return result
    if right and not left:
      result = np.load(self.file_path_s_1, allow_pickle=True)
      return result
    s_1 = np.load(self.file_path_s_1, allow_pickle=True)
    s_2 = np.load(self.file_path_s_2, allow_pickle=True)
    result = np.concatenate((s_1, s_2))
    return result

  def get_node_name(self):
    return self.id

##The Vantage Point Tree
The Vantage Point Tree is modeled as a class, that provides us the methods to:
- create the tree from a dataset
- search the k-nearest neighbors objects, given a query
- print the tree<br>

All these methods are based on recursion.<br>
The Vantage Point Tree can be created in two modes:
1. Test mode: in order to test the tree on very small datasets
2. Disk mode: in this mode we can handle very huge datasets, by storing the objects on disk and keeping in the tree just the reference to the files

### Save Tree on disk

The save method use the recursion. For each node, a json element is created, each element contains:
- node id
- boolean to recognize a leaf
- the pivot as numpy array

If we deal with an internal node, we have:
- left child id
- right child id

On the other hand, the leaf node has:
- path of the file containing the objects on the left
- path of the file containing the objects on the right

At the end we will have a json array, each entry represents a node.
This json is saved on disk.

### Load Tree from disk

The load method works in the opposite way with respect to save. 
It also uses recursion:
1. the algorithm loads and deserializes the json
2. it finds the node in the json array using the id (node has id "0")
3. it builds a node object from the json
4. it finds the left child through the left child id in the entry
5. it finds the right child through the right child id in the entry
6. it builds the nodes recursively until it reaches the leaves

In [75]:
class VP_Tree:

  def __init__(self, index_name, height, disk_mode=True, leaves_path=None, use_similarity=False):
    """by default the tree is built with euclidean distance and the leaves are saved on disk, 
    use_similarity=True allows you to use the cosine similarity
    use disk_mode=False if you want to keep all the tree in memory(not suggested for huge data)"""
    self.root = None
    self.index_name = index_name
    self.height = height #to review
    self.disk_mode = disk_mode
    self.leaves_path = leaves_path
    self.use_similarity=use_similarity
    self.distance_computed = 0
    self.file_accessed = 0
    self.file_created = 0

  def create_vptree(self, names_path, features_path):
    start = time.time()
    data = VP_Tree.read_data(names_path, features_path)
    n = len(data)
    print("Number of data:", n)
    max_height = math.floor(math.log(n,2)-1)
    print("The max height of the tree is:", max_height)
    if self.height > max_height: self.height = max_height
    self.distance_computed = 0
    #take 1 pivot randomly and set pivot as root
    self.root, s_1, s_2 = self.partition_by_median(data)
    print("Tree is building")
    self.create_tree_level(self.root, s_1, s_2, 1)
    end = time.time()
    print("Building of the tree completed in:", end-start, "s")
  
  def create_tree_level(self, node, s_1, s_2, iteration):
      is_leaf = iteration + 1 >= self.height
      left_node, s_1_left, s_2_left = self.partition_by_median(s_1, parent=node,is_left=True, is_leaf=is_leaf)
      right_node, s_1_right, s_2_right = self.partition_by_median(s_2, parent=node,is_left=False, is_leaf=is_leaf)
      node.add_children(right_node, left_node)
      if iteration + 1 < self.height:
        self.create_tree_level(left_node, s_1_left, s_2_left, iteration + 1)
        self.create_tree_level(right_node, s_1_right, s_2_right, iteration + 1)
      else:
        if self.disk_mode:
          left_path = self.get_leaves_path(left_node.get_node_name())
          right_path = self.get_leaves_path(right_node.get_node_name())
          left_node.save_leaf_objects_on_disk(left_path, s_1_left, s_2_left)
          right_node.save_leaf_objects_on_disk(right_path, s_1_right, s_2_right)
        else:
          left_node.add_objects(s_1_left, s_2_left)
          right_node.add_objects(s_1_right, s_2_right)

  def partition_by_median(self, data, parent=None,is_left=False,is_leaf=False):
    pivot_index = random.choice(range(len(data)))
    pivot = data[pivot_index]
    del data[pivot_index]
    #compute all the distances
    distances = np.array([self.compute_distance(pivot[1],element[1]) for element in data])
    #sort the distances
    zipped_data_distances = sorted(zip(data, distances), key= lambda x:x[1])
    ordered_data, distances = zip(*zipped_data_distances)
    median = np.median(distances)
    #get the median
    s_1 = [element for element, distance in zipped_data_distances if distance <= median]
    s_2 = [element for element, distance in zipped_data_distances if distance >= median]
    #update node
    if parent == None:
      node = Node(id="0", is_leaf=is_leaf, pivot=pivot, median=median)
    else:
      node_id = parent.id + str(0 if is_left else 1)
      node = Node(node_id, is_leaf=is_leaf, pivot=pivot, median=median)
    return node, s_1, s_2

  def save_vptree(file_path, tree):
    if not os.path.exists(file_path): os.mkdir(file_path)
    file = os.path.join(file_path, tree.index_name + '.json')
    if os.path.exists(file):
      os.remove(file)
    with open(file, 'a') as json_file:
      index_json = {"index": tree.index_name, "nodes":[], 
                    "height":tree.height, "use_similarity": tree.use_similarity}
      VP_Tree.save_node(tree.root, index_json)
      vp_tree_json = json.dumps(index_json, cls=NumpyEncoder)
      json_file.write(vp_tree_json)
      print("File saved correctly in:", file)
    return file
  
  def save_node(node, index_json):
    if node.is_leaf:
        row_json={"is_leaf":True, 
                    "id":node.id,
                    "pivot" : node.pivot,
                    "median":node.median, 
                    "left_file":node.file_path_s_1, 
                    "right_file":node.file_path_s_2}
        index_json["nodes"].append(row_json)
    else:
        row_json={"is_leaf":False,
                  "id":node.id, 
                  "pivot": node.pivot,
                  "median": node.median,
                  "right_child":node.right.id,
                  "left_child":node.left.id}
        index_json["nodes"].append(row_json)
        VP_Tree.save_node(node.left, index_json)
        VP_Tree.save_node(node.right,index_json)
    return

  def load_vptree(path):
    if not os.path.exists:
      print("the path do not exist")
      return None
    entry_list=[]
    with open(path,'r', encoding='utf-8') as f:
      json_tree = json.load(f)
      entry_list=json_tree["nodes"]
    root_node=VP_Tree.parse_node('0',entry_list)
    index_name = json_tree["index"]
    height = json_tree["height"]
    use_similarity = json_tree.get("use_similarity", False)
    vp_tree = VP_Tree(index_name=index_name,height=height,leaves_path=path, 
                      use_similarity=use_similarity)
    vp_tree.root = root_node
    print("Tree loaded correctly")
    return vp_tree

  def parse_node(id, nodes):
    node_json = None
    for element in nodes:
      if element["id"]==id:
        node_json = element
    node=Node(id=node_json["id"], is_leaf=node_json["is_leaf"], 
              pivot=node_json["pivot"], median=node_json["median"])
    if (node.is_leaf):
      node.file_path_s_1=node_json["left_file"]
      node.file_path_s_2=node_json["right_file"]
    else:
      right=VP_Tree.parse_node(node_json["right_child"],nodes)
      left=VP_Tree.parse_node(node_json["left_child"],nodes)
      node.add_children(left, right)
    return node

  def knn_search(self, k, query):
    start = time.time()
    nn = [None for i in range(k)]
    d_nn = [math.inf for i in range(k)]
    self.distance_computed = 0
    self.file_accessed = 0
    nn, d_nn = self.search_subtree(self.root, nn, d_nn, k, query)
    end = time.time()
    print("Query answered in", end-start, " s")
    return self.reorder_list_on_distances(nn, d_nn, desc=False)

  def search_subtree(self, node, nn, d_nn, k, query):
    pivot, median = node.pivot, node.median
    distance = self.compute_distance(pivot[1], query)
    if distance < d_nn[0]:
      d_nn[0] = distance
      nn[0] = pivot
      nn, d_nn = self.reorder_list_on_distances(nn, d_nn)
    if node.is_leaf:
      return self.search_in_leaf(node, nn, d_nn, k, query)
    if distance - d_nn[0] <= median:
      nn, d_nn = self.search_subtree(node.left, nn, d_nn, k, query)
    if distance + d_nn[0] >= median:
      nn, d_nn = self.search_subtree(node.right, nn, d_nn, k, query)
    return nn, d_nn

  def search_in_leaf(self, node, nn, d_nn, k, query):
    objects = []
    distance_pivot = self.compute_distance(node.pivot[1], query)
    left, right = False, False
    if self.disk_mode:
      if distance_pivot - d_nn[0] <= node.median: 
        left = True
        self.file_accessed = self.file_accessed + 1
      if distance_pivot + d_nn[0] >= node.median: 
        right = True
        self.file_accessed = self.file_accessed + 1
      objects = node.load_objects_from_disk(left=left, right=right)
    else:
      objects = node.objects_left + node.objects_right
    for obj in objects:
      distance = self.compute_distance(obj[1], query)
      if distance < d_nn[0]:
        nn[0] = obj
        d_nn[0] = distance
        nn, d_nn = self.reorder_list_on_distances(nn, d_nn)
    return nn, d_nn

  def reorder_list_on_distances(self, nn, d_nn, desc=True):
      zipped = sorted(zip(nn, d_nn), key= lambda x:x[1], reverse=desc)
      nn, d_nn = zip(*zipped)
      return list(nn), list(d_nn)

  def print_tree(node, level, disk_mode=True):
    indentation = "\n" + str(level * "\t")
    response = "id: " + node.id + " " + str(node.pivot)
    if node.is_leaf:
      if disk_mode: 
        response += indentation + str(node.file_path_s_1)
        response += indentation + str(node.file_path_s_2)
      else:
        response += indentation + str(node.objects_left)
        response += indentation + str(node.objects_right)
      return response
    response += indentation + VP_Tree.print_tree(node=node.right, level=level+1, disk_mode=disk_mode)
    response += indentation + VP_Tree.print_tree(node=node.left, level=level+1, disk_mode=disk_mode)
    return response

  def get_leaves_path(self, file_name):
    if not self.leaves_path is None:
      directory = os.path.join(self.leaves_path, self.index_name)
    else: directory = os.path.join(LEAF_FOLDER, self.index_name)
    if not os.path.exists(directory):
      os.mkdir(directory)
      print("directory created", directory)
    leaves_directory = os.path.join(directory, "leaves_"+ str(self.height))
    if not os.path.exists(leaves_directory):
      os.mkdir(leaves_directory)
    return os.path.join(leaves_directory, file_name)

  def compute_distance(self, a, b):
    self.distance_computed = self.distance_computed + 1
    if self.use_similarity:
      return 1 - np.dot(a,b)/(np.linalg.norm(a)*np.linalg.norm(b))
    return d.euclidean(a,b)

  def read_data(file_path_names, file_path_features):
    names = np.load(file_path_names)
    features = np.load(file_path_features)
    return [(name, feature) for name, feature in zip(names, features)]

#Deployment

## Index MN
This index has been created on the features extracted from Mobile Net

In [54]:
if __name__ == '__main__':
  vantage_point_tree = VP_Tree("index_mn",height=10, disk_mode=True, leaves_path=INDEX_DIR)
  vantage_point_tree.create_vptree(MN_ID, MN_FEATURES)

  index_name = vantage_point_tree.index_name
  dest_folder = os.path.join(INDEX_DIR, index_name)
  print("Destination Folder: ", dest_folder)

  VP_Tree.save_vptree(dest_folder,vantage_point_tree)

Number of data: 126000
The max height of the tree is: 15
Tree is building
directory created /content/gdrive/MyDrive/[MIRCV]FoodWebSearch/deployment/index_mn
Building of the tree completed in: 43.80826139450073 s
Destination Folder:  /content/gdrive/MyDrive/[MIRCV]FoodWebSearch/deployment/index_mn
File saved correctly in: /content/gdrive/MyDrive/[MIRCV]FoodWebSearch/deployment/index_mn/index_mn.json


In [60]:
if __name__ == '__main__':
  dest_folder = os.path.join(INDEX_DIR, "index_mn/index_mn.json")
  vantage_point_tree = VP_Tree.load_vptree(dest_folder)
  query = vantage_point_tree.root.pivot[1]
  print(vantage_point_tree.root.pivot[0])
  start = time.time()
  nn, d_nn = vantage_point_tree.knn_search(k=10, query=query)
  end = time.time()
  print("Results:", [element[0] for element in nn])
  print("Ricerca effettuata in:", end - start, "s")
  print("Distance Computed:", vantage_point_tree.distance_computed)
  print("File Accessed:", vantage_point_tree.file_accessed)
  print(d_nn)

Tree loaded correctly
991981.jpg
Query answered in 14.962364912033081  s
Results: ['991981.jpg', '432.jpg', '3696219.jpg', '2736144.jpg', '6201.jpg', '3345953.jpg', '3191742.jpg', '887401.jpg', '2909040.jpg', '3703331.jpg']
Ricerca effettuata in: 14.965189933776855 s
Distance Computed: 127487
File Accessed: 1024
[0.0, 17.095823399802523, 17.616884059335675, 18.024046354297713, 18.399501334960966, 18.84149145474033, 18.92051482612357, 18.948681193998965, 19.000750232911894, 19.044412773054002]


## Index Fine Tuned
This index has been created on the features extracted from the fine tuned deep network.

In [56]:
if __name__ == '__main__':
  vantage_point_tree = VP_Tree("index_fine_tuned",height=10, disk_mode=True, leaves_path=INDEX_DIR)
  vantage_point_tree.create_vptree(FINE_TUNED_ID, FINE_TUNED_FEATURES)

  index_name = vantage_point_tree.index_name
  dest_folder = os.path.join(INDEX_DIR, index_name)
  print("Destination Folder: ", dest_folder)

  VP_Tree.save_vptree(dest_folder,vantage_point_tree)

Number of data: 126000
The max height of the tree is: 15
Tree is building
directory created /content/gdrive/MyDrive/[MIRCV]FoodWebSearch/deployment/index_fine_tuned
Building of the tree completed in: 33.848634481430054 s
Destination Folder:  /content/gdrive/MyDrive/[MIRCV]FoodWebSearch/deployment/index_fine_tuned
File saved correctly in: /content/gdrive/MyDrive/[MIRCV]FoodWebSearch/deployment/index_fine_tuned/index_fine_tuned.json


In [61]:
if __name__ == '__main__':
  dest_folder = os.path.join(INDEX_DIR, "index_fine_tuned/index_fine_tuned.json")
  vantage_point_tree = VP_Tree.load_vptree(dest_folder)
  query = vantage_point_tree.root.pivot[1]
  print(vantage_point_tree.root.pivot[0])
  start = time.time()
  nn, d_nn = vantage_point_tree.knn_search(k=10, query=query)
  end = time.time()
  print("Results:", [element[0] for element in nn])
  print("Ricerca effettuata in:", end - start, "s")
  print("Distance Computed:", vantage_point_tree.distance_computed)
  print("File Accessed:", vantage_point_tree.file_accessed)

Tree loaded correctly
2719211.jpg
Query answered in 7.290550708770752  s
Results: ['2719211.jpg', '2400419.jpg', '1029917.jpg', '1029917.jpg', '2971476.jpg', '620631.jpg', '2268565.jpg', '1165903.jpg', '828774.jpg', '1881276.jpg']
Ricerca effettuata in: 7.295640707015991 s
Distance Computed: 127487
File Accessed: 1024
